In [2]:
!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 3.6 MB/s eta 0:00:0000:0100:01


In [ ]:
import numpy as np
import time
import scipy.io as sio
from numba import njit

@njit
def update_G(G, N, M, hz_hr, hr_hz, denom_edge, position_of_electrons):
    max_err = 0.0
    # n = 0
    for i in range(N):
        for j in range(1, M):
            a = G[j, i, 0]
            if i == 0:
                if j == position_of_electrons + 1:
                    G[j, 0, 0] = (4 * hz_hr * G[j, 1, 0] +
                                  hr_hz * (G[j - 1, 0, 0] + G[j + 1, 0, 0]) + 1) / denom_edge
                else:
                    G[j, 0, 0] = (4 * hz_hr * G[j, 1, 0] +
                                  hr_hz * (G[j - 1, 0, 0] + G[j + 1, 0, 0])) / denom_edge
            else:
                denom = 2 * i * (hz_hr + hr_hz)
                G[j, i, 0] = ((i - 0.5) * hz_hr * G[j, i - 1, 0] +
                              (i + 0.5) * hz_hr * G[j, i + 1, 0] +
                              i * hr_hz * (G[j - 1, i, 0] + G[j + 1, i, 0])) / denom
            err_val = abs(G[j, i, 0] - a)
            if err_val > max_err:
                max_err = err_val

    # n = 1 to N
    for n in range(1, N):
        for i in range(N):
            for j in range(1, M):
                a = G[j, i, n]
                if i == 0:
                    G[j, i, n] = (4 * hz_hr * G[j, i + 1, n] +
                                  hr_hz * (G[j - 1, i, n] + G[j + 1, i, n])) / denom_edge
                else:
                    denom = 2 * i * (hz_hr + hr_hz)
                    if j == position_of_electrons + 1 and i == n:
                        G[j, i, n] = ((i - 0.5) * hz_hr * G[j, i - 1, n] +
                                      (i + 0.5) * hz_hr * G[j, i + 1, n] +
                                      i * hr_hz * (G[j - 1, i, n] + G[j + 1, i, n]) + i) / denom
                    else:
                        G[j, i, n] = ((i - 0.5) * hz_hr * G[j, i - 1, n] +
                                      (i + 0.5) * hz_hr * G[j, i + 1, n] +
                                      i * hr_hz * (G[j - 1, i, n] + G[j + 1, i, n])) / denom
                err_val = abs(G[j, i, n] - a)
                if err_val > max_err:
                    max_err = err_val

    return max_err


def GreenSOLUT(R, H, N, M, position_of_electrons, eps, initialG):
    hr = R / N
    hz = H / M
    hz_hr = hz / hr
    hr_hz = hr / hz
    denom_edge = 4 * hz_hr + 2 * hr_hz

    G = np.copy(initialG)
    max_err = 1.0
    iteration = 0
    start_time = time.time()

    while max_err > eps:
        max_err = update_G(G, N, M, hz_hr, hr_hz, denom_edge, position_of_electrons)
        elapsed = time.time() - start_time
        print(f'Iter {iteration:5d} | max_err = {max_err:.3e} | time = {elapsed:.2f}s')
        iteration += 1

    return G


# === Parameters ===
R = 0.75  # cm
H = 0.20  # cm
N = 500
M = 200
position_of_electrons = 100
eps = 1e-4
initialG = np.zeros((M + 1, N + 1, N + 1), dtype=np.float32)  # reduce memory usage

# === Run Solver ===
G = GreenSOLUT(R, H, N, M, position_of_electrons, eps, initialG)

# === Save Result ===
filename = f'Green_R{str(R).replace(".", "pt")}_H{str(H).replace(".", "pt")}_N{N}_M{M}_posiE{position_of_electrons}_epsE{int(np.log10(eps))}.mat'
sio.savemat(filename, {'G': G})


Iter     0 | max_err = 2.308e-01 | time = 1.53s
Iter     1 | max_err = 1.086e-01 | time = 2.30s
Iter     2 | max_err = 6.797e-02 | time = 3.09s
Iter     3 | max_err = 4.636e-02 | time = 3.92s
Iter     4 | max_err = 3.214e-02 | time = 4.66s
Iter     5 | max_err = 2.687e-02 | time = 5.42s
Iter     6 | max_err = 2.303e-02 | time = 6.20s
Iter     7 | max_err = 1.996e-02 | time = 7.00s
Iter     8 | max_err = 1.733e-02 | time = 7.74s
Iter     9 | max_err = 1.514e-02 | time = 8.51s
Iter    10 | max_err = 1.333e-02 | time = 9.28s
Iter    11 | max_err = 1.189e-02 | time = 10.06s
Iter    12 | max_err = 1.106e-02 | time = 10.80s
Iter    13 | max_err = 1.031e-02 | time = 11.50s
Iter    14 | max_err = 9.608e-03 | time = 12.26s
Iter    15 | max_err = 8.966e-03 | time = 13.01s
Iter    16 | max_err = 8.381e-03 | time = 13.74s
Iter    17 | max_err = 7.848e-03 | time = 14.48s
Iter    18 | max_err = 7.363e-03 | time = 15.30s
Iter    19 | max_err = 6.921e-03 | time = 16.05s
Iter    20 | max_err = 6.519e-0

Iter   174 | max_err = 7.167e-04 | time = 144.62s
Iter   175 | max_err = 7.124e-04 | time = 145.51s
Iter   176 | max_err = 7.082e-04 | time = 146.32s
Iter   177 | max_err = 7.039e-04 | time = 147.12s
Iter   178 | max_err = 6.998e-04 | time = 147.96s
Iter   179 | max_err = 6.956e-04 | time = 148.71s
Iter   180 | max_err = 6.915e-04 | time = 149.46s
Iter   181 | max_err = 6.878e-04 | time = 150.21s
Iter   182 | max_err = 6.842e-04 | time = 150.96s
Iter   183 | max_err = 6.805e-04 | time = 151.71s
Iter   184 | max_err = 6.769e-04 | time = 152.41s
Iter   185 | max_err = 6.733e-04 | time = 153.17s
Iter   186 | max_err = 6.698e-04 | time = 154.03s
Iter   187 | max_err = 6.663e-04 | time = 154.81s
Iter   188 | max_err = 6.628e-04 | time = 155.71s
Iter   189 | max_err = 6.593e-04 | time = 156.47s
Iter   190 | max_err = 6.558e-04 | time = 157.31s
Iter   191 | max_err = 6.524e-04 | time = 158.15s
Iter   192 | max_err = 6.490e-04 | time = 158.97s
Iter   193 | max_err = 6.457e-04 | time = 159.73s


Iter   344 | max_err = 3.592e-04 | time = 283.54s
Iter   345 | max_err = 3.581e-04 | time = 284.32s
Iter   346 | max_err = 3.571e-04 | time = 285.36s
Iter   347 | max_err = 3.561e-04 | time = 286.39s
Iter   348 | max_err = 3.551e-04 | time = 287.21s
Iter   349 | max_err = 3.541e-04 | time = 287.97s
Iter   350 | max_err = 3.531e-04 | time = 288.72s
Iter   351 | max_err = 3.521e-04 | time = 289.49s
Iter   352 | max_err = 3.511e-04 | time = 290.28s
Iter   353 | max_err = 3.502e-04 | time = 291.05s
Iter   354 | max_err = 3.492e-04 | time = 291.81s
Iter   355 | max_err = 3.482e-04 | time = 292.53s
Iter   356 | max_err = 3.472e-04 | time = 293.54s
Iter   357 | max_err = 3.462e-04 | time = 294.46s
Iter   358 | max_err = 3.453e-04 | time = 295.27s
Iter   359 | max_err = 3.443e-04 | time = 296.10s
Iter   360 | max_err = 3.434e-04 | time = 297.02s
Iter   361 | max_err = 3.424e-04 | time = 297.94s
Iter   362 | max_err = 3.415e-04 | time = 298.82s
Iter   363 | max_err = 3.405e-04 | time = 299.67s


Iter   517 | max_err = 2.382e-04 | time = 430.37s
Iter   518 | max_err = 2.378e-04 | time = 431.32s
Iter   519 | max_err = 2.373e-04 | time = 432.26s
Iter   520 | max_err = 2.368e-04 | time = 433.21s
Iter   521 | max_err = 2.364e-04 | time = 434.11s
Iter   522 | max_err = 2.359e-04 | time = 434.98s
Iter   523 | max_err = 2.355e-04 | time = 435.83s
Iter   524 | max_err = 2.350e-04 | time = 436.74s
Iter   525 | max_err = 2.346e-04 | time = 437.71s
Iter   526 | max_err = 2.341e-04 | time = 438.71s
Iter   527 | max_err = 2.337e-04 | time = 439.51s
Iter   528 | max_err = 2.332e-04 | time = 440.33s
Iter   529 | max_err = 2.328e-04 | time = 441.09s
Iter   530 | max_err = 2.324e-04 | time = 441.85s
Iter   531 | max_err = 2.319e-04 | time = 442.56s
Iter   532 | max_err = 2.315e-04 | time = 443.28s
Iter   533 | max_err = 2.310e-04 | time = 444.01s
Iter   534 | max_err = 2.306e-04 | time = 444.80s
Iter   535 | max_err = 2.301e-04 | time = 445.76s
Iter   536 | max_err = 2.297e-04 | time = 446.64s


Iter   688 | max_err = 1.785e-04 | time = 573.20s
Iter   689 | max_err = 1.782e-04 | time = 574.12s
Iter   690 | max_err = 1.780e-04 | time = 575.07s
Iter   691 | max_err = 1.777e-04 | time = 576.04s
Iter   692 | max_err = 1.775e-04 | time = 576.94s
Iter   693 | max_err = 1.772e-04 | time = 577.92s
Iter   694 | max_err = 1.770e-04 | time = 578.80s
Iter   695 | max_err = 1.767e-04 | time = 579.67s
Iter   696 | max_err = 1.765e-04 | time = 580.50s
Iter   697 | max_err = 1.762e-04 | time = 581.40s
Iter   698 | max_err = 1.760e-04 | time = 582.30s
Iter   699 | max_err = 1.757e-04 | time = 583.17s
Iter   700 | max_err = 1.755e-04 | time = 584.10s
Iter   701 | max_err = 1.752e-04 | time = 584.99s
Iter   702 | max_err = 1.750e-04 | time = 585.78s
Iter   703 | max_err = 1.747e-04 | time = 586.54s
Iter   704 | max_err = 1.745e-04 | time = 587.34s
Iter   705 | max_err = 1.742e-04 | time = 588.11s
Iter   706 | max_err = 1.740e-04 | time = 588.87s
Iter   707 | max_err = 1.738e-04 | time = 589.67s


In [ ]:
for eps in [1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
    print(f"\n=== Solving for eps = {eps:.0e} ===")
    start = time.time()
    G = GreenSOLUT(R, H, N, M, position_of_electrons, eps, G)
    elapsed = time.time() - start
    print(f"Done in {elapsed:.2f} seconds")

    #saving
    eps_log10 = int(np.log10(eps))
    GreenFuncName = f"Green_R{str(R).replace('.', 'pt')}_H{str(H).replace('.', 'pt')}_N{N}_M{M}_posiE{position_of_electrons}_epsE{eps_log10}.mat"
    sio.savemat(GreenFuncName, {'G': G})
    print(f"Saved: {GreenFuncName}")